## Exploratory data analysis (EDA) of behvaioral data

Load necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append("src")

from src.load_files import load_posts_ratings

###  Resolve subject IDs

Option 1: Using BIDS IDs by going over the BIDS tree

In [ ]:
import sys
sys.path.append("src")
import importlib
from src import resolve_ids
importlib.reload(sys.modules['src.resolve_ids'])
from src.resolve_ids import attach_bids_ids_to_behav

In [ ]:
BIDS_ROOT       = "/path/to/bids/dataset"
BEHAV_DATA_PATH = "/path/to/behavioral_data.csv"
OUTPUT_PATH     = "/path/to/output_with_bids_ids.csv"

In [ ]:
df = attach_bids_ids_to_behav(
    bids_root=BIDS_ROOT,
    behav_csv=BEHAV_DATA_PATH,
    output_csv=OUTPUT_PATH,
    csv_session_col="session_id",
)

Option 2: Using manifest file

In [ ]:
from src import resolve_ids
importlib.reload(sys.modules['src.resolve_ids'])
from src.resolve_ids import merge_behavior_with_manifest_by_session

In [ ]:
MANIFEST_TSV    = "/path/to/bids/dataset"
BEHAV_DATA_PATH = "/path/to/behavioral_data.csv"
OUTPUT_PATH     = "/path/to/output_with_bids_ids.csv"

In [ ]:
df = merge_behavior_with_manifest_by_session(
    manifest_tsv=MANIFEST_TSV,
    behavioral_csv=BEHAV_DATA_PATH,
    output_csv=OUTPUT_PATH
)

###  Analyze e-prime behavioral data

Loads e-prime txt files including subject ratings of political stimuli

In [ ]:
from pathlib import Path
import sys
sys.path.append("src")
from src.eprime_ratings import build_long_df

In [ ]:
# Loading all e-prime txt files and converting to long format csv of ratings (by subject code 00XX -
# TODO: integrate with bids subject code)

data_dir = Path("path/to/behavioral_data_fmri")

df = build_long_df(
    data_dir,
    questionnaire_prefix="Potilical_views_questionnaire")

df.head()
out = data_dir / "political_views_ratings_long.csv"
df.to_csv(out, index=False)
out